In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleanss/clean.csv


In [2]:
df=pd.read_csv('/kaggle/input/cleanss/clean.csv')

In [3]:
df.head()

,Unnamed: 0,review,sentiment
0,0,one reviewer mentioned watching oz episode you...,1
1,1,wonderful little production filming technique ...,1
2,2,thought wonderful way spend time hot summer we...,1
3,3,basically there family little boy jake think t...,0
4,4,petter matteis love time money visually stunni...,1


In [4]:
df.drop(columns=['Unnamed: 0'], inplace= True)

In [5]:
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode you...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically there family little boy jake think t...,0
4,petter matteis love time money visually stunni...,1


In [6]:
df.shape

(50000, 2)

In [7]:
df.duplicated().sum()

423

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
df=df.drop_duplicates()

In [10]:
df.duplicated().sum()

0

In [12]:
df['sentiment'].value_counts()

sentiment
1    24882
0    24695
Name: count, dtype: int64

In [13]:
# Calculate word count for each review
df['word_count'] = df['review'].apply(lambda x: len(str(x).split()))

# Calculate the average word count
avg_word_count = df['word_count'].mean()

# Find the review with the maximum word count
max_word_count = df['word_count'].max()
max_review = df[df['word_count'] == max_word_count]['review'].values[0]

# Display the results
print(f"Average word count: {avg_word_count}")
print(f"Maximum word count: {max_word_count}")
print(f"Review with maximum word count: \n{max_review}")


Average word count: 119.4285253242431
Maximum word count: 1408
Review with maximum word count: 
match tag team table match bubba ray spike dudley v eddie guerrero chris benoit bubba ray spike dudley started thing tag team table match eddie guerrero chris benoit according rule match opponent go table order get win benoit guerrero heated early taking turn hammering first spike bubba ray german suplex benoit bubba took wind dudley brother spike tried help brother referee restrained benoit guerrero ganged corner benoit stomping away bubba guerrero set table outside spike dashed ring somersaulted top rope onto guerrero outside recovering taking care spike guerrero slipped table ring helped wolverine set tandem set double superplex middle rope would put bubba table spike knocked table right brother came crashing guerrero benoit propped another table corner tried irish whip spike bubba dashed blocked brother bubba caught fire lifted opponent back body drop bubba slammed guerrero spike stomped

In [14]:
import pandas as pd
from sklearn.utils import resample

# Assuming your dataframe has a 'review' column and a 'sentiment' column
# Sentiment 1: Positive, Sentiment 0: Negative

# Separate majority and minority classes
df_majority = df[df['sentiment'] == 1]
df_minority = df[df['sentiment'] == 0]

# Resample to balance the classes
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # sample without replacement
                                   n_samples=len(df_minority),  # match minority class
                                   random_state=42)  # reproducibility

# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Shuffle the data
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)


In [18]:
import spacy
import time

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Define the tokenizer function
def spacy_tokenizer(text):
    doc = nlp(text)
    return [token.text for token in doc if not token.is_stop and not token.is_punct]

# Start the timer
start_time = time.time()

# Apply the tokenizer to the 'review' column
df_balanced['tokens'] = df_balanced['review'].apply(spacy_tokenizer)

# End the timer
end_time = time.time()

# Calculate the total time taken
time_taken = end_time - start_time

print(f"Tokenization completed in {time_taken:.2f} seconds.")


Tokenization completed in 1007.72 seconds.


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example usage:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_balanced['review'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df_balanced['review'])

# Pad the sequences to ensure they are all of the same length
padded_sequences = pad_sequences(sequences, maxlen=200)


In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import f1_score
from keras.callbacks import Callback

# Define the tokenizer and fit it on your text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_balanced['review'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df_balanced['review'])

# Set the max_len (e.g., 200 or based on your data)
max_len = 200  # You can adjust this based on your data

# Pad the sequences to ensure uniform length
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Prepare the input (X) and target (y) variables
X = padded_sequences
y = df_balanced['sentiment']  # Assuming the sentiment column is your target

# Vocabulary size (+1 for padding index)
vocab_size = len(tokenizer.word_index) + 1

# Define the LSTM model
model = Sequential()

# Embedding layer: input_dim is vocab_size, output_dim is the dimensionality of embeddings
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))

# LSTM layer with Dropout
model.add(LSTM(64))
model.add(Dropout(0.5))  # 50% dropout

# Dense layer for output (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Custom callback to compute F1 Score
class F1Score(Callback):
    def __init__(self, validation_data):
        super(F1Score, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = (self.model.predict(X_val) > 0.5).astype("int32")
        f1 = f1_score(y_val, y_pred)
        print(f" - f1_score: {f1:.4f}")

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model with the F1Score callback
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val), callbacks=[F1Score(validation_data=(X_val, y_val))])


Epoch 1/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - f1_score: 0.8812
618/618 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.7842 - loss: 0.4436 - val_accuracy: 0.8840 - val_loss: 0.2840
Epoch 2/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - f1_score: 0.8818
618/618 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9040 - loss: 0.2455 - val_accuracy: 0.8803 - val_loss: 0.2851
Epoch 3/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - f1_score: 0.8756
618/618 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9245 - loss: 0.1978 - val_accuracy: 0.8789 - val_loss: 0.3094
Epoch 4/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - f1_score: 0.8782
618/618 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9391 - loss: 0.1614 - val_accuracy: 0.8797 - val_loss: 0.3323
Epoch 5/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 - f1_score: 0.8793
618/618 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9481 - loss: 0.1379 - val_accuracy: 0.8791 - val_loss: 0.3526


In [49]:
import numpy as np

# Your example reviews (5 positive and 5 negative)
reviews = [
    "The movie was fantastic! I loved every moment of it.",  # Positive
    "An amazing film with great storytelling and characters.",  # Positive
    "What a delightful experience! Highly recommended.",  # Positive
    "Incredible visuals and a wonderful plot. Must watch!",  # Positive
    "This movie made me feel so good! Great performances.",  # Positive
    "Worst movie ever. I regretted watching it.",  # Negative
    "It was a complete waste of time. So boring.",  # Negative
    "I did not enjoy this film at all. Very disappointing.",  # Negative
    "The acting was terrible and the story was nonsensical.",  # Negative
    "Absolutely not worth the hype. I hated it."  # Negative
]

# Tokenize and pad the reviews
example_sequences = tokenizer.texts_to_sequences(reviews)
padded_examples = pad_sequences(example_sequences, maxlen=max_len)

# Make predictions
predictions = model.predict(padded_examples)

# Display predictions, showing only the relevant probability
for i, review in enumerate(reviews):
    prob_positive = predictions[i][0]  # Probability of the positive class
    prob_negative = 1 - prob_positive  # Calculate probability of the negative class
    sentiment = "Positive" if prob_positive > 0.5 else "Negative"
    
    if sentiment == "Positive":
        print(f"Review: {review}\nPrediction: {sentiment} (Positive Probability: {prob_positive:.4f})\n")
    else:
        print(f"Review: {review}\nPrediction: {sentiment} (Negative Probability: {prob_negative:.4f})\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Review: The movie was fantastic! I loved every moment of it.
Prediction: Positive (Positive Probability: 0.8817)

Review: An amazing film with great storytelling and characters.
Prediction: Positive (Positive Probability: 0.8929)

Review: What a delightful experience! Highly recommended.
Prediction: Positive (Positive Probability: 0.9839)

Review: Incredible visuals and a wonderful plot. Must watch!
Prediction: Positive (Positive Probability: 0.8709)

Review: This movie made me feel so good! Great performances.
Prediction: Positive (Positive Probability: 0.7956)

Review: Worst movie ever. I regretted watching it.
Prediction: Negative (Negative Probability: 0.9595)

Review: It was a complete waste of time. So boring.
Prediction: Negative (Negative Probability: 0.9866)

Review: I did not enjoy this film at all. Very disappointing.
Prediction: Negative (Negative Probability: 0.8597)

Review: The acting was terrible and the story was nonsensical.
Predi